In [1]:
import numpy as np 
import pandas as pd
import time
import os

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

# Logistic regression class

In [2]:
class LogisticRegression(object):
    def __init__(self,
                 data_path='data/',  # WRITE THE PATH TO YOUR DATA HERE
                 optimizer='gd',
                 gd_lr=0.001,
                 eps=1e-8):
        # Save the optimizer
        self.optimizer = optimizer

        # Save hyperparameter settings
        self.gd_lr = gd_lr  # gd learning rate
        self.eps = eps  # epsilon, for numerical stability

        # Load the data
        self.df = pd.read_csv(os.path.join(data_path, 'bank/bank-full.csv'), sep=";")

        # Encoding data
        cols = self.df.columns
        num_cols = self.df._get_numeric_data().columns
        cat_cols = list(set(cols) - set(num_cols))

        ordinal_encoder = OrdinalEncoder()
        self.df[cat_cols] = ordinal_encoder.fit_transform(self.df[cat_cols])

        # split data
        self.train_data, self.test_data = train_test_split(self.df, test_size=0.2, random_state=1)
        self.train_data, self.val_data = train_test_split(self.train_data, test_size=0.25, random_state=1)

        # labels
        self.train_labels = self.train_data.y
        self.test_labels = self.test_data.y
        self.val_labels = self.val_data.y

        # drop the target value
        self.train_data = self.train_data.drop(['y'], axis=1)
        self.test_data = self.test_data.drop(['y'], axis=1)
        self.val_data = self.val_data.drop(['y'], axis=1)

        # transform back to numpy array
        self.train_data = self.train_data.to_numpy()
        self.test_data = self.test_data.to_numpy()
        self.val_data = self.val_data.to_numpy()
        self.train_labels = self.train_labels.to_numpy()
        self.test_labels = self.test_labels.to_numpy()
        self.val_labels = self.val_labels.to_numpy()

        # Prepend a vector of all ones to each of the dataset
        self.train_data = np.hstack([np.ones_like(self.train_data[:, 0])[:, np.newaxis], self.train_data])
        self.val_data = np.hstack([np.ones_like(self.val_data[:, 0])[:, np.newaxis], self.val_data])
        self.test_data = np.hstack([np.ones_like(self.test_data[:, 0])[:, np.newaxis], self.test_data])

        # Initialize the weight matrix
        np.random.seed(seed=42)
        self.start_w = np.random.rand(self.train_data.shape[1])

        # Initialize the train logs
        self.train_logs = {'train_accuracy': [], 'validation_accuracy': [], 'train_loss': [], 'validation_loss': []}

    def sigmoid(self, a):
        """
        inputs:
            v: float

        returns:
            the logistic sigmoid evaluated at a
        """
        # WRITE CODE HERE
        return 1 / (1 + np.exp(-a))

    def forward(self, w, X):
        """
        inputs: w: an array of the current weights, of shape (d,)
                X: an array of n datapoints, of shape (n, d)

        outputs: an array of the output of the logistic regression (not 0s and 1s yet)
        """
        # WRITE CODE HERE
        return self.sigmoid(np.dot(X, w))

    def loss(self, w, X, y):
        """
        inputs: w: an array of the current weights, of shape (d,)
                X: an array of n datapoints, of shape (n, d)

        outputs: the loss. This is exactly the negative log likelihood
        """
        # WRITE CODE HERE
        # Note: add self.eps to a value before taking its log
        E = 10 ** (-8)
        y_pred = self.sigmoid(np.dot(X, w))
        cost = -np.sum(y * np.log(y_pred + E) + (1 - y) * np.log(1 - y_pred + E))
        return cost

    def gradient(self, w, X, y):
        """
        inputs:
            w: an array of the current weights

        returns:
            an array representing the gradient of the loss
        """
        # WRITE CODE HERE
        grad = np.dot(X.T, (self.sigmoid(np.dot(X, w)) - y))
        return grad

    def gd_step(self, w, X, y):
        """
        inputs:
            w: an array of the current weights

        returns:
            a vector of weights updated according to a step of gradient descent
            on the whole train dataset, using the learning rate self.gd_lr
        """
        # WRITE CODE HERE
        grad = self.gradient(w, X, y)
        wnext = w - self.gd_lr * grad
        return wnext

    def compute_average_loss_and_accuracy(self, w, X, y):
        outputs = self.forward(w, X)
        predictions = np.array(np.round(outputs), dtype=int)
        accuracy = np.mean(y == predictions)
        loss = self.loss(w, X, y) / X.shape[0]
        return loss, accuracy, predictions

    def predict(self, X, w):
        y_predict = self.forward(X, w)
        return y_predict

    def train_loop(self, n_epochs):
        w = np.array(np.copy(self.start_w), dtype=np.float128)
        X = self.train_data
        y = self.train_labels

        # Choose an optimizer
        opt_step = self.gd_step

        for epoch in range(n_epochs):
            # GD
            w = opt_step(w, X, y)

            train_loss, train_accuracy, _ = self.compute_average_loss_and_accuracy(w, self.train_data,
                                                                                   self.train_labels)
            valid_loss, valid_accuracy, _ = self.compute_average_loss_and_accuracy(w, self.val_data, self.val_labels)

            self.train_logs['train_accuracy'].append(train_accuracy)
            self.train_logs['validation_accuracy'].append(valid_accuracy)
            self.train_logs['train_loss'].append(train_loss)
            self.train_logs['validation_loss'].append(valid_loss)
            
            print("train_accuracy:", train_accuracy, '   validation_accuracy: ', valid_accuracy, '   train_loss: ',train_loss, ' validation_loss: ' ,valid_loss )

        # y_predict = self.predict(X, w)

        return w

# main

In [3]:
if __name__ == "__main__":
    # WRITE CODE HERE
    # Instantiate, train, and evaluate your classifiers in the space below
    LR = LogisticRegression()
    start = time.time()
    LR.train_loop(100)
    end = time.time()
    print('Execution Time: ', end - start)

<ipython-input-2-6a71d502949a>:68: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-a))


train_accuracy: 0.8175919781759198    validation_accuracy:  0.8167440831674408    train_loss:  3.360079926981357567  validation_loss:  3.3756987302454180405
train_accuracy: 0.8174076531740765    validation_accuracy:  0.8166334881663349    train_loss:  3.3634753189952837568  validation_loss:  3.3777359654537737544
train_accuracy: 0.817149598171496    validation_accuracy:  0.8156381331563813    train_loss:  3.368228867814780423  validation_loss:  3.3960710823289751797
train_accuracy: 0.8163754331637544    validation_accuracy:  0.8146427781464278    train_loss:  3.38248951427327042  validation_loss:  3.414406199204176605
train_accuracy: 0.8137580181375802    validation_accuracy:  0.8115461181154612    train_loss:  3.4307040808710223168  validation_loss:  3.4714487850381365952
train_accuracy: 0.1759935117599351    validation_accuracy:  0.17827914178279142    train_loss:  15.178760449055639342  validation_loss:  15.1356593534978601575
train_accuracy: 0.816522893165229    validation_accuracy